# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv('../datasets/Orders.csv', index_col=0)
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
# let's group the amount spent by customerId
customers_amount_spent = orders[['CustomerID','Country','amount_spent']].groupby(['CustomerID','Country'],as_index=False).sum()

# there's some customers that bought from two different countries, therefore we will assign the country with the most amount spent.
duplicated_customerID = customers_amount_spent[customers_amount_spent.duplicated(subset='CustomerID')]['CustomerID']
duplicated_customers_countries = customers_amount_spent[customers_amount_spent['CustomerID'].isin(duplicated_customerID)].sort_values(by=['CustomerID','amount_spent'],ascending=[True,False])
duplicated_customers_countries

,CustomerID,Country,amount_spent
21,12370,Cyprus,3268.49
20,12370,Austria,277.20
40,12394,Denmark,891.40
39,12394,Belgium,381.08
58,12417,Belgium,3212.80
59,12417,Spain,436.30
64,12422,Switzerland,417.36
63,12422,Australia,386.20
72,12429,Denmark,3312.42
71,12429,Austria,437.98


In [4]:
#first we group by ID and add up the amount spent in order to get the total
grouped_customers_duplicated = duplicated_customers_countries.groupby('CustomerID', as_index=False).sum()
grouped_customers_duplicated

,CustomerID,amount_spent
0,12370,3545.69
1,12394,1272.48
2,12417,3649.10
3,12422,803.56
4,12429,3750.40
5,12431,6487.45
6,12455,2466.86
7,12457,2363.23


In [5]:
# second we sorted the customer_amount_spent by CustomerID and amount_spent:
customers_amount_spent = customers_amount_spent.sort_values(by=['CustomerID','amount_spent'], ascending=[True,False])
customers_amount_spent

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


In [6]:
# third, we drop duplicates by customer ID:

customers_amount_spent = customers_amount_spent.drop_duplicates('CustomerID',keep='first')
customers_amount_spent

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


In [7]:
#finally we assign the total calculated in grouped_customers_duplicated to the corresponding rows in customers_amount_spent

#first set index of grouped_customers_duplicated as CustomerID
grouped_customers_duplicated = grouped_customers_duplicated.set_index('CustomerID')

In [8]:
customers_amount_spent[customers_amount_spent['CustomerID'].isin(grouped_customers_duplicated.index)]

,CustomerID,Country,amount_spent
21,12370,Cyprus,3268.49
40,12394,Denmark,891.40
58,12417,Belgium,3212.80
64,12422,Switzerland,417.36
72,12429,Denmark,3312.42
74,12431,Australia,5514.67
96,12455,Cyprus,1698.90
100,12457,Switzerland,1970.53


In [9]:
for index,row in customers_amount_spent.iterrows():
    if row['CustomerID'] in grouped_customers_duplicated.index.to_list():
        print(row['CustomerID'], row['amount_spent'])
        customers_amount_spent.loc[index,'amount_spent'] = grouped_customers_duplicated.loc[row['CustomerID'],'amount_spent']
        print(row['CustomerID'], row['amount_spent'])

12370 3268.4899999999984
12370 3268.4899999999984
12394 891.4000000000001
12394 891.4000000000001
12417 3212.7999999999997
12417 3212.7999999999997
12422 417.36
12422 417.36
12429 3312.420000000001
12429 3312.420000000001
12431 5514.67
12431 5514.67
12455 1698.8999999999996
12455 1698.8999999999996
12457 1970.5299999999997
12457 1970.5299999999997


C:\Users\Pol\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
customers_amount_spent[customers_amount_spent['CustomerID'].isin(grouped_customers_duplicated.index)]

,CustomerID,Country,amount_spent
21,12370,Cyprus,3545.69
40,12394,Denmark,1272.48
58,12417,Belgium,3649.10
64,12422,Switzerland,803.56
72,12429,Denmark,3750.40
74,12431,Australia,6487.45
96,12455,Cyprus,2466.86
100,12457,Switzerland,2363.23


In [11]:
# let's get the amounts for the quantile 75% and the quantile 95%.
quantiles = customers_amount_spent['amount_spent'].quantile([0.75,0.95])
print(quantiles)
quantile75 = round(quantiles[0.75],3)
quantile95 = round(quantiles[0.95],3)
print(quantile75)
print(quantile95)

0.75    1661.640
0.95    5840.182
Name: amount_spent, dtype: float64
1661.64
5840.182


In [12]:
# now we get a data frame of CustomerIDs whose amount spent surpasses the 95% quantile calculated.
customersVIP_df = customers_amount_spent[customers_amount_spent.loc[:,'amount_spent'] >= quantile95]
# we transform the CustomerIDs found into a list, named customerVIP
customersVIP = customersVIP_df['CustomerID'].to_list()


# let's do the same for the Preferred customers:
customerspreferred_df = customers_amount_spent[(customers_amount_spent.loc[:,'amount_spent'] >= quantile75) & (customers_amount_spent.loc[:,'amount_spent'] < quantile95)]
customerspreferred = customerspreferred_df['CustomerID'].to_list()

In [13]:
# now we create a new column customers_amount_spent, labeled VIP_Preferred, and populate with the value 'Regular'
customers_amount_spent.loc[:,'VIP_Preferred'] = 'Regular'
customers_amount_spent['VIP_Preferred'].value_counts()

# finally we update the values when customer ID is in the corresponding lists of VIP or Preferred

customers_amount_spent.loc[customers_amount_spent['CustomerID'].isin(customersVIP), 'VIP_Preferred'] = 'VIP'
customers_amount_spent.loc[customers_amount_spent['CustomerID'].isin(customerspreferred), 'VIP_Preferred'] = 'Preferred'
customers_amount_spent['VIP_Preferred'].value_counts()

C:\Users\Pol\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Pol\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Regular      3254
Preferred     868
VIP           217
Name: VIP_Preferred, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [14]:
# we can filter first by VIP customers and then group by country.
customers_amount_spent_VIP = customers_amount_spent[customers_amount_spent['VIP_Preferred'] == 'VIP']
countries_VIP = customers_amount_spent_VIP.groupby('Country').count()['CustomerID'].sort_values(ascending=False).to_frame()
countries_VIP.columns=['VIP_Customers']
countries_VIP
#UK is the country with most VIP Customers

,VIP_Customers
Country,
United Kingdom,177
Germany,10
France,9
Switzerland,3
EIRE,2
Japan,2
Portugal,2
Spain,2
Australia,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [15]:
customers_amount_spent_VIPPreferred = customers_amount_spent[customers_amount_spent['VIP_Preferred'].
                                                             isin(['VIP','Preferred'])]
countries_VIPPreferred = customers_amount_spent_VIPPreferred.groupby('Country').count()['CustomerID'].sort_values(ascending=False).to_frame()
countries_VIPPreferred.columns=['VIPorPreferred_Customers']
countries_VIPPreferred
#again United Kingdom has the most combination of VIP and Preferred Customers

,VIPorPreferred_Customers
Country,
United Kingdom,932
Germany,39
France,29
Belgium,11
Switzerland,9
Spain,7
Portugal,7
Norway,7
Italy,5
